In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn-v0_8')

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [2]:
data = pd.read_csv('data/_train_sem09 (1).csv')

In [3]:
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

## Логистическая регрессия

In [10]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель
log_reg.fit(X_train, y_train)
#Выводим значения метрики
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


## Случайный лес

In [11]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.83


## GridSearchCV
### LogisticRegression

In [23]:
# зададим пространство поиска гиперпараметров
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
) 

#Обучаем модель
%time grid_search.fit(X_train, y_train)

#Выводим значения метрики 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
70 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
59 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Python\Python312\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Python\Python312\Lib\site-packages\sklearn\utils\_param_val

CPU times: total: 9.2 s
Wall time: 14min 34s
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


### RandomForest

In [26]:
# зададим пространство поиска гиперпараметров
param_grid = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth': list(np.linspace(15, 40, 5, dtype=int))
              }

grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)

#Обучаем модель
%time grid_search.fit(X_train, y_train)

#Выводим значения метрики 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 3.48 s
Wall time: 1min 34s
f1_score на тестовом наборе: 0.83
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_leaf': 5, 'n_estimators': 130}


## RandomizedSearchCV
### LogisticRegression

In [31]:
# зададим пространство поиска гиперпараметров
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 10, 10, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)

#Обучаем модель
%time random_search.fit(X_train, y_train)

#Выводим значения метрики
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrappe

CPU times: total: 5.62 s
Wall time: 8min 47s
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}


### RandomForest

In [32]:
# зададим пространство поиска гиперпараметров
param_distributions = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth': list(np.linspace(15, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1
)

#Обучаем модель
%time random_search_forest.fit(X_train, y_train)

#Выводим значения метрики
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

c:\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: total: 3.45 s
Wall time: 1min 40s
f1_score на тестовом наборе: 0.83
Наилучшие значения гиперпараметров: {'n_estimators': 130, 'min_samples_leaf': 5, 'max_depth': 15}


## Hyperopt

### LogisticRegression

In [109]:
# зададим пространство поиска гиперпараметров логистической регрессии
pen = ['l1', 'l2']
sol = ['liblinear', 'saga']
space_lr = {'penalty': hp.choice('penalty', pen),
              'solver': hp.choice('solver', sol),
               'C': hp.uniform('C', 0.01, 1)}

In [110]:
# зафиксируем random_state
random_state = 42

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver'], 
             'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=42, max_iter=1000)

    # обучаем модель
    model.fit(X, y)

    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [112]:
%%time
# начинаем подбор гиперпараметров

trials_lr = Trials() # используется для логирования результатов

best_lr=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_lr, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr))

  5%|▌         | 1/20 [00:05<01:40,  5.28s/trial, best loss: -0.771966158616668]

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 45%|████▌     | 9/20 [09:12<06:41, 36.46s/trial, best loss: -0.7833737934825968] 

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 65%|██████▌   | 13/20 [13:47<05:11, 44.44s/trial, best loss: -0.7839950244781424]

...

 70%|███████   | 14/20 [17:10<09:14, 92.48s/trial, best loss: -0.7839950244781424]

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 90%|█████████ | 18/20 [23:03<02:35, 77.94s/trial, best loss: -0.7839950244781424] 

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [26:37<00:00, 79.88s/trial, best loss: -0.7839950244781424] 
Наилучшие значения гиперпараметров {'C': 0.4857515970241288, 'penalty': 0, 'solver': 1}
CPU times: total: 7min 37s
Wall time: 26min 37s


In [138]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=1000,
    penalty=pen[best_lr['penalty']],
    solver=sol[best_lr['solver']],
    C=float(best_lr['C'])
)
model_lr.fit(X_train, y_train)
y_test_pred_lr = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

f1_score на тестовом наборе: 0.79


c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [114]:
best_lr=fmin(hyperopt_lr, # наша функция 
          space=space_lr, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials_lr, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_lr))

 40%|████      | 20/50 [00:00<?, ?trial/s, best loss=?]

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 42%|████▏     | 21/50 [03:10<1:32:13, 190.81s/trial, best loss: -0.7843672038523106]

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 44%|████▍     | 22/50 [06:28<1:30:59, 194.97s/trial, best loss: -0.7843672038523106]

...

 86%|████████▌ | 43/50 [55:02<07:04, 60.69s/trial, best loss: -0.7855542288452043] 

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 98%|█████████▊| 49/50 [1:02:50<01:01, 61.24s/trial, best loss: -0.7855542288452043]

c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 50/50 [1:06:08<00:00, 132.28s/trial, best loss: -0.7855542288452043]
Наилучшие значения гиперпараметров {'C': 0.41809334370290907, 'penalty': 0, 'solver': 1}


In [139]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(
    random_state=random_state,
    max_iter=1000,
    penalty=pen[best_lr['penalty']],
    solver=sol[best_lr['solver']],
    C=float(best_lr['C'])
)
model_lr.fit(X_train, y_train)
y_test_pred_lr = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

f1_score на тестовом наборе: 0.79


c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### RandomForest

In [116]:
# зададим пространство поиска гиперпараметров случайного леса
space_rf={'n_estimators': hp.quniform('n_estimators', 100, 300, 10),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [117]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)

    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [118]:
%%time
# начинаем подбор гиперпараметров

trials_rf = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=30, # максимальное количество итераций
          trials=trials_rf, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

100%|██████████| 30/30 [03:56<00:00,  7.88s/trial, best loss: -0.8118379122354072]
Наилучшие значения гиперпараметров {'max_depth': 21.0, 'min_samples_leaf': 2.0, 'n_estimators': 220.0}
CPU times: total: 1min 7s
Wall time: 3min 56s


In [119]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best_rf['n_estimators']),
    max_depth=int(best_rf['max_depth']),
    min_samples_leaf=int(best_rf['min_samples_leaf'])
)
model_rf.fit(X_train, y_train)
y_test_pred_rf = model_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

f1_score на тестовом наборе: 0.84


In [120]:
best_rf=fmin(hyperopt_rf, # наша функция 
          space=space_rf, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials_rf, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

100%|██████████| 50/50 [02:34<00:00,  7.73s/trial, best loss: -0.8118379122354072]
Наилучшие значения гиперпараметров {'max_depth': 21.0, 'min_samples_leaf': 2.0, 'n_estimators': 220.0}


In [121]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best_rf['n_estimators']),
    max_depth=int(best_rf['max_depth']),
    min_samples_leaf=int(best_rf['min_samples_leaf'])
)
model_rf.fit(X_train, y_train)
y_test_pred_rf = model_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

f1_score на тестовом наборе: 0.84


## Optuna
### LogisticRegression

In [140]:
pen = ['l1', 'l2']
sol = ['liblinear', 'saga']

def optuna_lr(trial, cv=5, X=X_train, y=y_train):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', pen)
  solver = trial.suggest_categorical('solver', sol)
  C = trial.suggest_float('C', 2, 10)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state,
                                          max_iter=1000)
  # обучаем модель
  model.fit(X, y)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

In [141]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_lr = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr.optimize(optuna_lr, n_trials=20)

[I 2024-05-23 19:51:08,938] A new study created in memory with name: LogisticRegression
c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-05-23 19:55:59,662] Trial 0 finished with value: 0.7640738320877152 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 4.5752710618796035}. Best is trial 0 with value: 0.7640738320877152.
[I 2024-05-23 19:56:07,565] Trial 1 finished with value: 0.764361394454163 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 9.419166864446495}. Best is trial 1 with value: 0.764361394454163.
c:\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-05-23 20:00:39,871] Trial 2 finished with value: 0.7634000007148927 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 9.110378207408214}. Best is

CPU times: total: 9min 38s
Wall time: 29min 33s


In [142]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(**study_lr.best_params,random_state=random_state, max_iter=1000)
model_lr.fit(X_train, y_train)
y_test_pred_lr = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

f1_score на тестовом наборе: 0.79


In [143]:
%%time
# можем прододжить подбор, указав n_trials(любое число, которое добавится к предыдущим итерациям) 
study_lr.optimize(optuna_lr, n_trials=30)

[I 2024-05-23 20:20:47,540] Trial 20 finished with value: 0.7677022694512562 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 5.018153917843323}. Best is trial 11 with value: 0.7707333750503635.
[I 2024-05-23 20:20:50,255] Trial 21 finished with value: 0.7688695063357643 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 2.232900652080941}. Best is trial 11 with value: 0.7707333750503635.
[I 2024-05-23 20:20:52,892] Trial 22 finished with value: 0.7691184402020911 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 2.1990366062589755}. Best is trial 11 with value: 0.7707333750503635.
[I 2024-05-23 20:20:55,725] Trial 23 finished with value: 0.7682864133145701 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 3.1432932514034375}. Best is trial 11 with value: 0.7707333750503635.
[I 2024-05-23 20:20:58,643] Trial 24 finished with value: 0.7685283966293417 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 2.6295804901366493}. Best i

CPU times: total: 4min 35s
Wall time: 12min 15s


In [144]:
# рассчитаем точность для тестовой выборки
model_lr = linear_model.LogisticRegression(**study_lr.best_params,random_state=random_state, max_iter=1000)

model_lr.fit(X_train, y_train)

y_test_pred_lr = model_lr.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

f1_score на тестовом наборе: 0.79


### RandomForest

In [147]:
def optuna_rf(trial, cv=5):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 1)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

In [148]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=20)

[I 2024-05-23 20:48:55,149] A new study created in memory with name: RandomForestClassifier
C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 1)
C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
[I 2024-05-23 20:49:06,949] Trial 0 finished with value: 0.8061445541975901 and parameters: {'n_estimators': 183, 'max_depth': 24, 'min_samples_leaf': 7

CPU times: total: 1min 6s
Wall time: 3min 20s


In [149]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=random_state)
model_rf.fit(X_train, y_train)
y_test_pred_rf = model_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

f1_score на тестовом наборе: 0.83


In [150]:
%%time
# можем прододжить подбор, указав n_trials(любое число, которое добавится к предыдущим итерациям) 
study_rf.optimize(optuna_rf, n_trials=30)

C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 1)
C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
C:\Users\anzel\AppData\Local\Temp\ipykernel_7520\1749906253.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
[I 2024-05-23 20:52:30,993] Trial 20 finished with value: 0.7962186263923653 and parameters: {'n_estimators': 272, 'max_depth': 37, 'min_samples_leaf': 10}. Best is trial 13 with value: 0.8175471608735325.
C:\Users\anzel\AppData\Local\Temp\ipyk

CPU times: total: 2min 5s
Wall time: 5min 32s


In [151]:
# рассчитаем точность для тестовой выборки
model_rf = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=random_state, )
model_rf.fit(X_train, y_train)
y_test_pred_rf = model_rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

f1_score на тестовом наборе: 0.83
